In [1]:
#!pip install -U transformers
#!pip install -U datasets
#!pip install optuna
import os
import sys
HOME = os.path.abspath('..')
sys.path.append(HOME)
os.chdir(HOME)
import pandas as pd
#!pip install transformers
from transformers import RobertaConfig, RobertaModel,RobertaForSequenceClassification, Trainer,AutoModelForSequenceClassification, EarlyStoppingCallback 
from transformers import AutoTokenizer
from transformers.models.roberta import RobertaPreTrainedModel
import torch
from torch import nn
from transformers import TrainingArguments
import glob
import optuna

In [2]:
MODEL_NAME =  "distilbert-base-uncased" #"roberta-base" 
TARGET_COL = 'averageRating'
MODEL_FOLDER = 'only_text_features'
FINAL_MODEL_NAME = f"{MODEL_NAME}-{TARGET_COL}"
CATEGORIES_AS_TEXT = False
NUMERIC_AS_TEXT = False
DATE_AS_TEXT = False
text_input_col = 'text_input'
COLAB = False
DEBUG = False

if COLAB == True:
  if not os.path.exists('data'):
    os.mkdir('data')
  if not os.path.exists('data/processed'):
    os.mkdir('data/processed')

  drive.mount('/content/gdrive/')
  for filename in glob.glob(os.path.join('gdrive/MyDrive/atdl', '*.*')):
      shutil.copy(filename, 'data/processed')

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def process_text_data(data_:pd.DataFrame,text_col,padding ="max_length", truncation = True, na_filler = ""):

    '''
    
    '''
    data = data_.copy()
    data[text_col] = data[text_col].fillna(na_filler)
    encodings = tokenizer(data[text_col].tolist(), padding=padding, truncation=truncation)
    return encodings
    

def columns_to_single_text(df,cols_to_transform,new_col_name = 'text_input',sep = tokenizer.sep_token,nan_replacement = tokenizer.unk_token ):

  '''
  
  Creates a new column called new_col_name with with all columns in cols_to_transform concatenated into a single text
  '''
  df[new_col_name] = df[cols_to_transform].astype(str).replace('nan',nan_replacement).agg(f' {sep} '.join, axis=1)


class NAFiller:

  def __init__(self,train):
    self.train = train

  def fit(self,column = 'Budget',groupby=['top_genre','top_country']):
    self.mapping = self.train.groupby(groupby)[column].median().reset_index()
    self.mapping = self.mapping.rename(columns={column:'na_filler'})
    self.median = self.train[column].median()
    self.column=column


  def transform(self,test,round = False):
    self.na_filler = test.merge(self.mapping,how='left')['na_filler']
    self.na_filler = self.na_filler.fillna(self.median)

    test[self.column] = test[self.column].reset_index(drop=True).fillna(self.na_filler).values

    if round:
      test[self.column].round().astype(int)
      


  def fit_transform(self,test,column = 'Budget',groupby=['top_genre','top_country']):
    self.fit(column,groupby)
    self.transform()
    self.column=column
        

def create_dataset_split(split,text_cols,text_input_col,TARGET_COL):

  #If all columns in text_cols are combined into a single text. A n
  columns_to_single_text(split,text_cols)

  #Get split encodings
  split_encodings = process_text_data(split,text_input_col)

  #get labels
  split_labels = split[TARGET_COL].tolist()

  #Create dataset objects
  split_dataset = IMDbDataset(split_encodings, split_labels)

  return split_dataset

In [5]:
all_cols =  ['Budget',
             'averageRating',
             'cast',
             'countries',
             'director',
             'genres',
             'imdb_id',
             'languages',
             'overview',
             'production companies',
             'release_date',
             'revenue_worldwide_BOM',
             'runtimeMinutes',
             'title']

categoric_cols = ['cast',
                  'countries',
                  'director',
                  'genres',
                  'languages',
                  'production companies']

text_cols = ['title','overview']                  
date_cols = ['release_date']
numeric_cols = ['Budget','runtimeMinutes']

if CATEGORIES_AS_TEXT:
  text_cols+=categoric_cols

if NUMERIC_AS_TEXT:
  text_cols+=numeric_cols

if DATE_AS_TEXT:
  text_cols+=date_cols


train_ids = pd.read_csv('data/processed/train.csv',usecols=['imdb_id'])['imdb_id'].tolist()
val_ids = pd.read_csv('data/processed/val.csv',usecols=['imdb_id'])['imdb_id'].tolist()
test_ids = pd.read_csv('data/processed/test.csv',usecols=['imdb_id'])['imdb_id'].tolist()
df = pd.read_csv('data/processed/df.csv',usecols = all_cols,parse_dates=['release_date']).sample(frac=1) #shuffle


df[categoric_cols] = df[categoric_cols].apply(lambda x: x.str.replace('|',', '),axis=0) #Change pipe to comma, its more meaningful

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [6]:
#Additional auxilary columns
df['top_genre'] = df['genres'].apply(lambda x: x.split(', ')[0])
df['top_country'] = df['countries'].apply(lambda x: x.split(', ')[0] if isinstance(x,str) else x)
df['year'] = df['release_date'].dt.year

In [7]:
#Create splits
if DEBUG:
    train = df[df['imdb_id'].isin(train_ids)].sample(frac=0.2)
    val = df[df['imdb_id'].isin(val_ids)].sample(frac=0.2)
    test = df[df['imdb_id'].isin(test_ids)]
else:
    train = df[df['imdb_id'].isin(train_ids)]
    val = df[df['imdb_id'].isin(val_ids)]
    test = df[df['imdb_id'].isin(test_ids)]


#Fill na in some columns with statistics
naf = NAFiller(train)
naf.fit(column = 'Budget',groupby=['top_genre','top_country'])
naf.transform(train,round=True)
naf.transform(val,round=True)
naf.transform(test,round=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
'''
#If all columns in text_cols are combined into a single text. A n
columns_to_single_text(train,text_cols)
columns_to_single_text(val,text_cols)
columns_to_single_text(test,text_cols)


#Get train encodings
train_encodings = process_text_data(train,text_input_col)
val_encodings = process_text_data(val,text_input_col)
test_encodings = process_text_data(test,text_input_col)

#get labels
train_labels = train[TARGET_COL].tolist()
val_labels = val[TARGET_COL].tolist()
test_labels = test[TARGET_COL].tolist()

#Create dataset objects
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)
'''


train_dataset=create_dataset_split(train,text_cols,text_input_col,TARGET_COL)
val_dataset=create_dataset_split(val,text_cols,text_input_col,TARGET_COL)
test_dataset=create_dataset_split(test,text_cols,text_input_col,TARGET_COL)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
def get_model():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                         problem_type='regression',
                                                         num_labels=1
def get_model_by_name(model_name):
    return AutoModelForSequenceClassification.from_pretrained(model_name,
                                                         problem_type='regression',
                                                         num_labels=1
                                                        )                                                )

from itertools import product
import numpy as np
from pprint import pprint

epochs = 10
num_evals = 20
patience = 2 if DEBUG else 30
callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)]
eval_steps = 50 if DEBUG else 100

hparams = {'batch_size' : [8,16,32],
           'learning_rate' : [1e-5, 2e-5, 3e-5,5e-5],
           'weight_decay' : [0.1,0.01],
           'repeats': range(1)}

combs = list(product(*[range(len(i)) for i in list(hparams.values())]))
scores = np.zeros([len(i) for i in list(hparams.values())])
best_score = float('inf')


trials_df = []

for idx,comb_indexes in enumerate(combs):
    comb_values = {name:val[idx] for name,val,idx in zip(hparams.keys(),hparams.values(),comb_indexes)}
    
    
    print('training with following hparams:')
    pprint(comb_values)
    
    training_args = TrainingArguments(output_dir=f"{MODEL_NAME}-{TARGET_COL}",
                                      per_device_train_batch_size = comb_values['batch_size'],
                                      learning_rate=comb_values['learning_rate'],
                                      weight_decay=comb_values['weight_decay'],
                                      seed = 42,
                                      fp16=True,
                                      per_device_eval_batch_size = 16,
                                      warmup_ratio=0.06,
                                      num_train_epochs = epochs,
                                      evaluation_strategy = "steps",
                                      save_strategy = "steps",
                                      load_best_model_at_end=True,
                                      eval_steps = eval_steps,
                                      save_steps = eval_steps,
                                      save_total_limit = 1,
                                      log_level = 'error',
                                      disable_tqdm = True
                                      
                                    )

    trainer = Trainer(
        model_init=get_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks = callbacks
    )
    
    trainer.train()

    score = trainer.evaluate()['eval_loss']
    scores[tuple(comb_indexes)] = score
    
    comb_values['score'] = score
    trials_df.append(comb_values)
    
    if score<best_score:
        print(f'got a better model, with score {np.round(score,4)} saving...')
        best_score = score
        trainer.save_model(f'models/{MODEL_FOLDER}/{FINAL_MODEL_NAME}')
        print('saved')
        

trials_df = pd.DataFrame(rows)
trials_df.to_csv(f'models/{MODEL_FOLDER}/hparams_trials.csv',index=False)
    
'''
    
training_args = TrainingArguments(output_dir=f"{MODEL_NAME}-{TARGET_COL}",
                                  seed = 42,
                                  fp16=True,
                                  per_device_eval_batch_size = 16,
                                  warmup_ratio=0.06,
                                  num_train_epochs = epochs,
                                  evaluation_strategy = "steps",
                                  save_strategy = "steps",
                                  load_best_model_at_end=True,
                                  eval_steps = eval_steps,
                                  save_steps = eval_steps,
                                  save_total_limit = 3
                                 )

trainer = Trainer(
    model_init=get_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks = callbacks
)


#Grid search
search_space = {'learning_rate' : [1e-5, 2e-5, 3e-5, 5e-5],
                'weight_decay' : [0.1,0.01],
                'per_device_train_batch_size' : [32,16,8]
               }




def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_categorical("learning_rate",search_space["learning_rate"]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", search_space["per_device_train_batch_size"]),
        "weight_decay": trial.suggest_categorical("weight_decay", search_space["weight_decay"])
    }


best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,
                                         n_trials=None,
                                         sampler = optuna.samplers.GridSampler(search_space),
                                         study_name = 'imdb_rating_finetune',
                                         direction='minimize',
                                         pruner=optuna.pruners.PatientPruner(None,patience=patience)
                                        )
'''


training with following hparams:
{'batch_size': 8, 'learning_rate': 1e-05, 'repeats': 0, 'weight_decay': 0.1}


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


{'eval_loss': 34.42953109741211, 'eval_runtime': 9.5585, 'eval_samples_per_second': 183.502, 'eval_steps_per_second': 11.508, 'epoch': 0.1}
{'eval_loss': 14.13512897491455, 'eval_runtime': 9.7196, 'eval_samples_per_second': 180.461, 'eval_steps_per_second': 11.317, 'epoch': 0.2}
{'eval_loss': 2.718254566192627, 'eval_runtime': 9.8907, 'eval_samples_per_second': 177.338, 'eval_steps_per_second': 11.122, 'epoch': 0.29}
{'eval_loss': 1.279853343963623, 'eval_runtime': 9.9863, 'eval_samples_per_second': 175.641, 'eval_steps_per_second': 11.015, 'epoch': 0.39}
{'loss': 15.0569, 'learning_rate': 8.032520325203253e-06, 'epoch': 0.49}
{'eval_loss': 1.317325234413147, 'eval_runtime': 10.1072, 'eval_samples_per_second': 173.54, 'eval_steps_per_second': 10.883, 'epoch': 0.49}
{'eval_loss': 1.1985527276992798, 'eval_runtime': 10.3331, 'eval_samples_per_second': 169.745, 'eval_steps_per_second': 10.645, 'epoch': 0.59}
{'eval_loss': 1.2244387865066528, 'eval_runtime': 10.3961, 'eval_samples_per_seco

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


{'eval_loss': 34.429317474365234, 'eval_runtime': 10.4166, 'eval_samples_per_second': 168.385, 'eval_steps_per_second': 10.56, 'epoch': 0.1}
{'eval_loss': 14.136425971984863, 'eval_runtime': 10.4336, 'eval_samples_per_second': 168.11, 'eval_steps_per_second': 10.543, 'epoch': 0.2}
{'eval_loss': 2.715629816055298, 'eval_runtime': 10.2708, 'eval_samples_per_second': 170.775, 'eval_steps_per_second': 10.71, 'epoch': 0.29}
{'eval_loss': 1.2795828580856323, 'eval_runtime': 10.3315, 'eval_samples_per_second': 169.772, 'eval_steps_per_second': 10.647, 'epoch': 0.39}
{'loss': 15.0474, 'learning_rate': 8.048780487804879e-06, 'epoch': 0.49}
{'eval_loss': 1.3251065015792847, 'eval_runtime': 10.3194, 'eval_samples_per_second': 169.971, 'eval_steps_per_second': 10.66, 'epoch': 0.49}
{'eval_loss': 1.1514538526535034, 'eval_runtime': 10.3115, 'eval_samples_per_second': 170.101, 'eval_steps_per_second': 10.668, 'epoch': 0.59}
{'eval_loss': 1.2293764352798462, 'eval_runtime': 10.3607, 'eval_samples_per

...

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


{'eval_loss': 1.4554216861724854, 'eval_runtime': 10.3072, 'eval_samples_per_second': 170.173, 'eval_steps_per_second': 10.672, 'epoch': 0.39}
{'eval_loss': 1.0725258588790894, 'eval_runtime': 10.3392, 'eval_samples_per_second': 169.646, 'eval_steps_per_second': 10.639, 'epoch': 0.78}
{'eval_loss': 1.3536043167114258, 'eval_runtime': 10.348, 'eval_samples_per_second': 169.502, 'eval_steps_per_second': 10.63, 'epoch': 1.17}
{'eval_loss': 1.1303107738494873, 'eval_runtime': 10.3907, 'eval_samples_per_second': 168.804, 'eval_steps_per_second': 10.586, 'epoch': 1.56}
{'loss': 4.5732, 'learning_rate': 4.28927680798005e-05, 'epoch': 1.95}
{'eval_loss': 1.1339911222457886, 'eval_runtime': 10.3417, 'eval_samples_per_second': 169.605, 'eval_steps_per_second': 10.637, 'epoch': 1.95}
{'eval_loss': 1.232366919517517, 'eval_runtime': 10.3811, 'eval_samples_per_second': 168.961, 'eval_steps_per_second': 10.596, 'epoch': 2.34}
{'eval_loss': 1.085791826248169, 'eval_runtime': 10.3723, 'eval_samples_pe

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


{'eval_loss': 1.4519883394241333, 'eval_runtime': 10.3287, 'eval_samples_per_second': 169.819, 'eval_steps_per_second': 10.65, 'epoch': 0.39}
{'eval_loss': 1.0716913938522339, 'eval_runtime': 10.3598, 'eval_samples_per_second': 169.308, 'eval_steps_per_second': 10.618, 'epoch': 0.78}
{'eval_loss': 1.2223597764968872, 'eval_runtime': 10.277, 'eval_samples_per_second': 170.672, 'eval_steps_per_second': 10.704, 'epoch': 1.17}
{'eval_loss': 1.1179624795913696, 'eval_runtime': 10.3573, 'eval_samples_per_second': 169.349, 'eval_steps_per_second': 10.621, 'epoch': 1.56}
{'loss': 4.5706, 'learning_rate': 4.28927680798005e-05, 'epoch': 1.95}
{'eval_loss': 1.1000239849090576, 'eval_runtime': 10.4041, 'eval_samples_per_second': 168.587, 'eval_steps_per_second': 10.573, 'epoch': 1.95}
{'eval_loss': 1.1460316181182861, 'eval_runtime': 10.3716, 'eval_samples_per_second': 169.116, 'eval_steps_per_second': 10.606, 'epoch': 2.34}
{'eval_loss': 1.1451269388198853, 'eval_runtime': 10.3632, 'eval_samples_

'\n    \ntraining_args = TrainingArguments(output_dir=f"{MODEL_NAME}-{TARGET_COL}",\n                                  seed = 42,\n                                  fp16=True,\n                                  per_device_eval_batch_size = 16,\n                                  warmup_ratio=0.06,\n                                  num_train_epochs = epochs,\n                                  evaluation_strategy = "steps",\n                                  save_strategy = "steps",\n                                  load_best_model_at_end=True,\n                                  eval_steps = eval_steps,\n                                  save_steps = eval_steps,\n                                  save_total_limit = 3\n                                 )\n\ntrainer = Trainer(\n    model_init=get_model,\n    args=training_args,\n    train_dataset=train_dataset,\n    eval_dataset=val_dataset,\n    callbacks = callbacks\n)\n\n\n#Grid search\nsearch_space = {\'learning_rate\' : [1e-5, 2e-5, 3

In [11]:
'''
rows=[]
for idx,comb_indexes in enumerate(combs):
    row = {name:val[idx] for name,val,idx in zip(hparams.keys(),hparams.values(),comb_indexes)}
    row['score']=scores[tuple(comb_indexes)]
    rows.append(row)
trials_df = pd.DataFrame(rows)
trials_df.to_csv(f'models/{MODEL_FOLDER}/hparams_trials.csv',index=False)
'''

In [41]:
#Test set performance
best_model = get_model_by_name(f'models/{MODEL_FOLDER}/{FINAL_MODEL_NAME}/')
trainer_best_model = Trainer(model=best_model)
test_stats_best_model = trainer_best_model.evaluate(test_dataset)
pd.DataFrame([test_stats_best_model]).to_csv(f'models/{MODEL_FOLDER}/test_stats_best_model.csv'
                                             ,index=False)

loading configuration file models/only_text_features/distilbert-base-uncased-averageRating/config.json
Model config DistilBertConfig {
  "_name_or_path": "models/only_text_features/distilbert-base-uncased-averageRating/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "regression",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file models/only_text_features/distilbert-base-uncased-averageRating/pytorch_model.bin
All model checkpoint weights were used when in